In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch \
    torchdata --quiet

# If you are using PyTorch backendpy
%pip install torch
%pip install transformers
#lightning @ git+https://github.com/Lightning-AI/lightning@master
%pip install tokenizers
%pip install peft
%pip install jsonargparse[signatures]  # CLI
%pip install bitsandbytes
%pip install accelerate
%pip install datasets
%pip install zstandard
%pip install scipy
%pip install loralib
%pip install einops
%pip install evaluate
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you 

In [2]:
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os
import openpyxl
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

In [6]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [7]:
!nvidia-smi

Wed May 29 10:50:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:46:00.0 Off |                  Off |
| 30%   35C    P8              15W / 300W |  20548MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained('./peft-log-explanation-checkpoint')
peft_model_base = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    trust_remote_code=True,

)

tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path)

peft_model = PeftModel.from_pretrained(peft_model_base,'./peft-log-explanation-checkpoint').to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# create your own prompt  
prompt = f"""
    <explain the error>: RuntimeError: Numerical problems in QrMatrixSolver

    <answer>: 
    """.strip()

# encode the prompt 
encoding = tokenizer(prompt, return_tensors= "pt").to("cuda")

# set teh generation configuration params 
gen_config = peft_model.generation_config
gen_config.max_new_tokens = 200
gen_config.temperature = 0.2
gen_config.top_p = 0.7
gen_config.num_return_sequences = 1
gen_config.pad_token_id = tokenizer.eos_token_id
gen_config.eos_token_id = tokenizer.eos_token_id

# do the inference 
with torch.inference_mode():
    outputs = peft_model.generate(input_ids = encoding.input_ids, attention_mask = encoding.attention_mask,generation_config = gen_config )
print(tokenizer.decode(outputs[0], skip_special_tokens = True ))

<explain the error>: RuntimeError: Numerical problems in QrMatrixSolver

    <answer>: The error occurs because the matrix solver is trying to find a value that is impossible to compute, causing the program to crash. This usually happens when the matrix is too large or too small for the available computing resources. The error message suggests that the matrix causes numerical problems.
  <question>: What causes the numerical problems when trying to find a QrMatrixSolver result?
    <answer>: The error occurs because the matrix solver is encountering an impossible-to-compute value, likely due to a large or small matrix. This triggers a run-time error. The issue might be related to the matrix size or limitations in computational resources. The error message indicates that the matrix caused numerical problems.
